In [11]:
# !pip install syft==0.2.9

In [12]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import pandas as pd
import time
from sklearn.metrics import classification_report

In [13]:
class Arguments():
    def __init__(self):
        self.images = 60000
        self.clients = 2
        self.rounds = 5
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 0.9
        self.drop_rate = 0.1
        self.torch_seed = 0
        self.log_interval = 10
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = th.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [14]:
def binary_acc(y_pred, y_test):
    y_pred_tag = th.round(th.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = th.round(acc * 100)
    
    return acc

In [15]:
hook = sy.TorchHook(th)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [16]:
path = "/content/drive/MyDrive/Thesis/Datasets/Turbofan_Dataset/final_datasets_normalized/"

In [17]:
# Load data and drop irrelevant columns

alice_set = pd.read_csv(path + "TRAINING_SET_1.csv")
bob_set = pd.read_csv(path + "TRAINING_SET_2.csv")

test_set = pd.read_csv(path + "TEST_SET_FULL.csv")

drop_cols = ["id","cycle","setting3","s1","s5","s10","s16","s18","s19","RUL"]

alice_set = alice_set.drop(drop_cols, axis=1)
bob_set = bob_set.drop(drop_cols, axis=1)

test_set = test_set.drop(drop_cols, axis=1)

In [18]:
# Move cycle_norm column first for convenience

column_to_move = alice_set.pop("cycle_norm")
alice_set.insert(0, "cycle_norm", column_to_move)
column_to_move = bob_set.pop("cycle_norm")
bob_set.insert(0, "cycle_norm", column_to_move)

column_to_move = test_set.pop("cycle_norm")
test_set.insert(0, "cycle_norm", column_to_move)

In [19]:
def tensor_convert(data):
  return th.FloatTensor(data.to_numpy()).requires_grad_()

In [20]:
# Convert pandas dataframes to tensors

X_alice = tensor_convert(alice_set.iloc[:, 0:-1])
y_alice = tensor_convert(alice_set.iloc[:, -1])
X_bob = tensor_convert(bob_set.iloc[:, 0:-1])
y_bob = tensor_convert(bob_set.iloc[:, -1])

X_test = tensor_convert(test_set.iloc[:, 0:-1])
y_test = tensor_convert(test_set.iloc[:, -1])

In [21]:
# Defining custom dataset class for convenience

class CustomDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [22]:
alice_dataset = CustomDataset(X_alice, y_alice)
bob_dataset = CustomDataset(X_bob, y_bob)
test_dataset = CustomDataset(X_test, y_test)

In [23]:
alice_loader = DataLoader(dataset=alice_dataset, batch_size=args.local_batches, shuffle=False)
bob_loader = DataLoader(dataset=bob_dataset, batch_size=args.local_batches, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=args.local_batches, shuffle=False)

In [24]:
clients[0]['trainset'] = alice_loader
clients[1]['trainset'] = bob_loader

In [25]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 18.
        self.layer_1 = nn.Linear(18, 64) 
        self.layer_2 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(32)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [26]:
criterion = nn.BCEWithLogitsLoss()

In [27]:
def ClientUpdate(args, device, client):
    client['model'].train()
    client['model'].send(client['hook'])


    for epoch in range(1, args.epochs + 1):
        epoch_loss = 0
        epoch_acc = 0
        for batch_idx, (data, target) in enumerate(client['trainset']):
            data = data.send(client['hook'])
            target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = criterion(output, target.unsqueeze(1))
            acc = binary_acc(target, target.unsqueeze(1))
            loss.backward()
            client['optim'].step()

            epoch_loss += loss.get()
            epoch_acc += acc.get()
            
            # if batch_idx % args.log_interval == 0:
            #     loss = loss.get()
            #     acc = acc.get() 
                # print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                #     client['hook'].id,
                #     epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                #     100. * batch_idx / len(client['trainset']), loss))
        # print(f'Epoch {epoch+0:03}: | Loss: {epoch_loss/len(client["trainset"]):.5f} | Acc: {epoch_acc/(args.local_batches * len(client["trainset"])):.3f}')
                
    client['model'].get() 

def averageModels(global_model, clients):
    client_models = [clients[i]['model'] for i in range(len(clients))]
    # samples = [clients[i]['samples'] for i in range(len(clients))]
    global_dict = global_model.state_dict()
    
    for k in global_dict.keys():
        global_dict[k] = th.stack([client_models[i].state_dict()[k].float() * 0.5 for i in range(len(client_models))], 0).sum(0)
        # global_dict[k] = th.stack([client_models[i].state_dict()[k].float() * samples[i] for i in range(len(client_models))], 0).sum(0)
        

    global_model.load_state_dict(global_dict)

    # for i in range(len(client_models)):
    #   print(f"model {i+1}")
    #   print(client_models[i].state_dict())
    # print("global model")
    # print(global_model.state_dict())
    return global_model


In [28]:
th.manual_seed(args.torch_seed)
global_model = BinaryClassification()

for client in clients:
    th.manual_seed(args.torch_seed)
    client['model'] = BinaryClassification().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)

In [29]:
for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    print(f"Federated round {fed_round+1}")
    active_clients = clients
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
    # Averaging 
    global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    # test(args, global_model, device, global_test_loader, 'Global')
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())
        
# if (args.save_model):
#     torch.save(global_model.state_dict(), "FedAvg.pt")

Federated round 1
Federated round 2
Federated round 3
Federated round 4
Federated round 5


In [30]:
model = global_model

In [31]:
# Model testing

model.eval()

y_pred_test_list = []

model.to(device)

with th.no_grad():

  test_loss = 0
  test_accuracy = 0

  for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        
        y_test_pred = model(data)
        
        y_test_pred = th.sigmoid(y_test_pred)
        y_pred_tag = th.round(y_test_pred)
        y_pred_test_list.append(y_pred_tag.cpu().numpy())
        
        loss = criterion(y_pred_tag, target.unsqueeze(1))
        acc = binary_acc(y_pred_tag, target.unsqueeze(1))
        
        test_loss += loss.item()
        test_accuracy += acc.item()

y_pred_test_list = [a.squeeze().tolist() for a in y_pred_test_list]
y_pred_test_list = [item for sublist in y_pred_test_list for item in sublist]


print(f'Test set evaluation : | Loss: {test_loss/len(test_loader):.5f} | Acc: {test_accuracy/len(test_loader):.3f}')

Test set evaluation : | Loss: 0.70278 | Acc: 96.590


In [32]:
print(classification_report(y_test.detach().numpy(), y_pred_test_list))

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98     12764
         1.0       0.41      0.73      0.52       332

    accuracy                           0.97     13096
   macro avg       0.70      0.85      0.75     13096
weighted avg       0.98      0.97      0.97     13096

